In [44]:
import numpy as np
from scipy.optimize import brentq, fsolve
import matplotlib.pyplot as plt

In [2]:
def x_prime(x, a):
    return -x[0] + a*x[1] + (x[0]**2)*x[1]

In [3]:
def y_prime(x, y, a, b):
    return b - a*y - (x**2)*y

In [4]:
def func(x, a, b):
    return [a - x[0] - (4*x[0]*x[1])/(1 + x[0]**2), b*x[0] * (1 - x[1]/(1+x[0]**2))]

In [5]:
def get_fixed_point(a, b):
    solution = fsolve(func, [0, 0], args=(a, b))
    return solution

In [13]:
def get_jacobian(x, y, a, b):
    j11 = -1 - (4*y / (1+x**2)) + ((8*y*x**2) / ((1+x**2)**2))
    j12 = -4*x / (1+x**2)
    j21 = b - b*y / (1+x**2) + 2*b*y*x**2 / (1+x**2)**2
    j22 = -b*x / (1+x**2)
    return np.array([[j11, j12], [j21, j22]])

In [16]:
def get_stability(j):
    tr = np.trace(j)
    det = np.linalg.det(j)
    is_spiral = 4*det > tr
    is_stable = tr < 0 and det > 0
    return is_spiral, is_stable

In [34]:
a_vec = np.linspace(0, 50, 50)
b_vec = np.linspace(0, 50, 50)

In [36]:
roots = np.zeros([a_vec.size, b_vec.size, 2])
spirals = np.zeros([a_vec.size, b_vec.size]).astype(bool)
stability = np.zeros([a_vec.size, b_vec.size]).astype(bool)
for i, a in enumerate(a_vec):
    for j, b in enumerate(b_vec):
        root = get_fixed_point(a, b)
        roots[i, j, :] = root
        jacobian = get_jacobian(root[0], root[1], a, b)
        spirals[i, j], stability[i, j] = get_stability(jacobian)
        

C:\Users\Luya\AppData\Local\Temp\ipykernel_5424\139466760.py:2: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  solution = fsolve(func, [0, 0], args=(a, b))


In [43]:
stable_spirals = np.logical_and(spirals, stability)
stable_spiral_points = []

plt.scatter()

[[False False False ... False False False]
 [False  True  True ... False False False]
 [False  True  True ... False False False]
 ...
 [False False False ...  True  True  True]
 [False False False ...  True  True  True]
 [False False False ...  True  True  True]]
